In [1]:
import os
import pandas as pd
import janitor
from tqdm.notebook import tqdm

from utilities import read_json

# Use this to figure out getting the keys in json
# https://www.youtube.com/watch?v=rWcLDax-VmM&t=3s

# Google console for PyPI data
# https://console.cloud.google.com/marketplace/product/gcp-public-data-pypi/pypi?q=search&referrer=search
# https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=pypi&page=dataset

# Schema here
# https://packaging.python.org/en/latest/guides/analyzing-pypi-package-downloads/#data-schema

pd.set_option('display.max_columns', 100)

output_path = "../output/cache_pypi_downloads_payload"

In [2]:
data_list = []
for filename in tqdm(os.listdir(output_path)):
    if filename.endswith(".json"):
        filename = filename.replace(".json", "")
        _, date, _ = filename.split("_", maxsplit=3)
        
        filepath = os.path.join(output_path, filename)
        data = read_json(filepath)
        downloads = data["rows"][0]["download_count"]
        package = data["args"][-1]
        
        data_list.append([package, date, downloads])
        pass

  0%|          | 0/29234 [00:00<?, ?it/s]

In [3]:
# Read in 100 random sample
df_random_sample = (
    pd.read_csv("../input/random_sample.csv")
    .assign(slug=lambda df: df["github"].str.replace("https://github.com/", ""))
    .assign(slug=lambda df: df["slug"].apply(lambda x: x[:-1] if x.endswith("/") else x))
    .assign(slug=lambda df: df["slug"].str.lower().str.strip())
    .assign(fileslug=lambda df: df["slug"].str.replace("/", "_"))
    .assign(treated=1)
    # Assign the two types
    .assign(boughtstars=lambda df: (df.index < 25).astype(int))
    .assign(treated2=lambda df: df["boughtstars"] + df["treated"])
)
df_random_sample.head(3)

,pkg,return_code,github_url,homepage,earliest_release,gh_url_check,github,pypi,label,slug,fileslug,treated,boughtstars,treated2
0,bird-ospf-link-db-parser,200.0,https://github.com/Andrew-Dickinson/bird-ospf-...,NaN,2023-04-29T07:23:44,1.0,https://github.com/Andrew-Dickinson/bird-ospf-...,https://pypi.org/project/bird-ospf-link-db-par...,bird-ospf-link-db-parser\r\nhttps://github.com...,andrew-dickinson/bird-ospf-link-db-parser,andrew-dickinson_bird-ospf-link-db-parser,1,1,2
1,asciicli,200.0,https://github.com/mrq-andras/asciicli,https://github.com/mrq-andras/asciicli,2023-04-28T07:22:55,1.0,https://github.com/mrq-andras/asciicli,https://pypi.org/project/asciicli/#history,asciicli\r\nhttps://github.com/mrq-andras/asci...,mrq-andras/asciicli,mrq-andras_asciicli,1,1,2
2,bdpotentiometer,200.0,https://github.com/bond-anton/BDPotentiometer,https://github.com/bond-anton/BDPotentiometer,2023-04-27T06:35:18,1.0,https://github.com/bond-anton/BDPotentiometer,https://pypi.org/project/bdpotentiometer/#history,bdpotentiometer\r\nhttps://github.com/bond-ant...,bond-anton/bdpotentiometer,bond-anton_bdpotentiometer,1,1,2


In [4]:
df = (
    pd.DataFrame(data_list, columns=["pkg", "date", "download_count"])
    .assign(date=lambda df: pd.to_datetime(df["date"]))
    .sort_values(["pkg", "date"], ignore_index=True)
    .assign(tt_downloads=lambda df: df.groupby("pkg")["download_count"].cumsum())
    # Merge to get treatment status
    .merge(
        df_random_sample.select_columns("pkg", "treated", "boughtstars", "treated2"),
        how="left",
        on="pkg",
        validate="m:1",
        indicator=True,
    )    
    .assign(treated=lambda df: df["treated"].fillna(0).apply(int))
    .assign(boughtstars=lambda df: df["boughtstars"].fillna(0).apply(int))
    .assign(treated2=lambda df: df["treated2"].fillna(0).apply(int))
    .assign(date=lambda df: pd.to_datetime(df["date"]))    
)
# Assert that 100 repos are assigned as treated
assert 100 == len(df.drop_duplicates("pkg").query("treated==1"))
assert 25 == len(df.drop_duplicates("pkg").query("treated2==2"))
df

,pkg,date,download_count,tt_downloads,treated,boughtstars,treated2,_merge
0,a-pandas-ex-df2htmlstring,2023-04-25,0,0,0,0,0,left_only
1,a-pandas-ex-df2htmlstring,2023-04-26,0,0,0,0,0,left_only
2,a-pandas-ex-df2htmlstring,2023-04-27,0,0,0,0,0,left_only
3,a-pandas-ex-df2htmlstring,2023-04-28,7,7,0,0,0,left_only
4,a-pandas-ex-df2htmlstring,2023-04-29,7,14,0,0,0,left_only
...,...,...,...,...,...,...,...,...
29229,zoomaker,2023-06-06,0,135,0,0,0,left_only
29230,zoomaker,2023-06-07,1,136,0,0,0,left_only
29231,zoomaker,2023-06-08,1,137,0,0,0,left_only
29232,zoomaker,2023-06-09,0,137,0,0,0,left_only


In [5]:
df.to_csv("../output/pkg_pypi_downloads.csv", index=False)